In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from langchain.llms import WatsonxLLM
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
import langchain
import ast
# from Generate_Response import GenerateLLMResponse

import warnings
warnings.filterwarnings("ignore")

In [2]:
from typing import Any, Optional
from uuid import UUID

from langchain.callbacks.base import BaseCallbackHandler

from genai import Client, Credentials
from genai.extensions.langchain import LangChainInterface
from genai.text.generation import (
    DecodingMethod,
    ModerationHAP,
    ModerationParameters,
    TextGenerationParameters,
)

# make sure you have a .env file under genai root with
# GENAI_KEY=<your-genai-key>
# GENAI_API=<genai-api-endpoint> (optional) DEFAULT_API = "https://bam-api.res.ibm.com"
load_dotenv()


def heading(text: str) -> str:
    """Helper function for centering text."""
    return "\n" + f" {text} ".center(80, "=") + "\n"


print(heading("Generate text with langchain"))


class Callback(BaseCallbackHandler):
    def on_llm_new_token(
        self,
        token: str,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> Any:
        print(f"Token received: {token}")


llm = LangChainInterface(
    # model_id="ibm/granite-13b-chat-v2",
    model_id='mistralai/mistral-7b-instruct-v0-2',
    client=Client(credentials=Credentials.from_env()),
    parameters=TextGenerationParameters(
        decoding_method=DecodingMethod.GREEDY,
        max_new_tokens=200,
        min_new_tokens=1,
        temperature=0.5,
        top_k=50,
        top_p=1,
        repetition_penalty=1
    ),
    moderations=ModerationParameters(
        # Threshold is set to very low level to flag everything (testing purposes)
        # or set to True to enable HAP with default settings
        hap=ModerationHAP(input=True, output=True, threshold=0.01)
    ),
)

prompt = "Tell me about IBM."
print(f"Prompt: {prompt}")

result = llm.generate(prompts=[prompt], callbacks=[Callback()])

print(f"Answer: {result.generations[0][0].text}")
print(result.llm_output)
print(result.generations[0][0].generation_info)

ImportError: Could not import langchain: Please install ibm-generative-ai[langchain] extension.

In [ ]:
os.listdir('data/')

['ms-marco-200-rows.csv', 'state_of_the_union.txt']

In [4]:
df = pd.read_csv("data/ms-marco-200-rows.csv")
df.head()

,question,contexts,ground_truths
0,walgreens store sales average,['The average Walgreens salary ranges from app...,"['Approximately $15,000 per year.']"
1,how much do bartenders make,['A bartender’s income is comprised mostly of ...,"['$21,550 per year'\n 'The average hourly wage..."
2,what is a furuncle boil,"['Knowledge center. A boil, also known as a fu...","['A boil, also called a furuncle, is a deep fo..."
3,what can urinalysis detect,['Urinalysis: One way to test for bladder canc...,"['Detect and assess a wide range of disorders,..."
4,what is vitamin a used for,['Since vitamin A is fat-soluble it is not nee...,"['Shigellosis, diseases of the nervous system,..."


In [5]:
df.contexts.iloc[0], df.ground_truths.iloc[0]

("['The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.'\n 'The average revenue in 2011 of a Starbuck Store was $1,078,000, up  from $1,011,000 in 2010.    The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).'\n 'In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The average size for a typical Walgreens is about 14,500 square feet and the sales floor averages about 11,000 square feet. How do we select locations for

In [6]:
df["ground_truths"] = [ast.literal_eval(gt) for gt in df["ground_truths"] ]
df["contexts"] = [ast.literal_eval(ctx) for ctx in df["contexts"]]

df.contexts.iloc[0], df.ground_truths.iloc[0]

(['The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.The average revenue in 2011 of a Starbuck Store was $1,078,000, up  from $1,011,000 in 2010.    The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The average size for a typical Walgreens is about 14,500 square feet and the sales floor averages about 11,000 square feet. How do we select locations for new stores

In [7]:
df.question.iloc[0]

'walgreens store sales average'

In [8]:
def text_to_chunks(texts: str, chunk_length: int = 100, chunk_overlap: int = 25) -> list:
    """
    Splits the text into equally distributed chunks with 25-word overlap.
    Args:
        texts (str): Text to be converted into chunks.
        chunk_length (int): Maximum number of words in each chunk.
        chunk_overlap (int): Number of words to overlap between chunks.
    """
    words = texts.split(' ')
    n = len(words)
    chunks = []
    chunk_number = 1
    i = 0
    while i < n:  # Corrected the length check
        chunk = words[i: min(i + chunk_length, n)]
        i = i + chunk_length - chunk_overlap
        #print(len(chunk))
        chunk = ' '.join(chunk).strip()
        chunks.append({"text": chunk})
        chunk_number += 1
    return chunks

In [9]:
df['chunks'] = df['contexts'].apply(lambda x: [i['text'] for i in text_to_chunks(x[0])])
df['chunks'][0]

['The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.The average revenue in 2011 of a Starbuck Store was $1,078,000, up  from $1,011,000 in 2010.    The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.',
 'The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The average size for a typical Walgreens is about 14,5

In [10]:
data = list()
for i in range(len(df)):
    for j in df['chunks'][i]:
        doc = Document(
            metadata={
                "question": df['question'][i],
            },
            page_content=j)
        data.append(doc)

In [4]:
# Initialie the embedding model
model_name = "intfloat/e5-large-v2"
model_kwargs = {'device': 'cpu'}

embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs
)

In [12]:
# Putting the data into vector store
vectorstore = FAISS.from_documents(data, embeddings_model)

# Retrieve similar documents to hypothetical answer from the vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}, verbose = False)

In [13]:
# Testing a sample query from the dataset
query = "what is the average water pressure for a home"
docs = retriever.get_relevant_documents(query,verbose=False)

# Print the context
for i in docs:
    print(i.page_content)

come in handy when your home's water pressure becomes too high or low. You should also test to make sure your water pressure regulator is doing its job and this is the exact procedure to do that. I hope this video helps. Normal Pressure Range. The normal range of water pressure in a residential application is between 40 psi and 80 psi. There is no specific rule about how much pressure is best for your house, since everyone's needs are different. However, anything below 40 psi will likely lead to poor performance of devices that use water. Anything higher than
pressure is measured in pounds per square inch (PSI), and normal water pressure is typically between 30 and 80 PSI. Functional flow is the volume of water flowing through your pipes and arriving at individual fixtures. This is a bell-shaped device that reduces the water pressure. Aim for water pressure that is 60-70 PSI. If the house has low pressure, you first want to determine if the house is on a public water supply system or a

In [14]:
from llama_index.prompts import PromptTemplate

template = (
    "[INST] \n"
    "Context: {context} \n"
    "- Take the context above and use that to answer questions in a detailed and professional way. \n"
    "- if you dont know the answer just say ” i dont know “. \n"
    "- refrain from using any other knowledge other than the text provided. \n"
    "- don't mention that you are answering from the text, impersonate as if this is coming from your knowledge \n"
    "- For the questions whose answer is not available in the provided context, just say please ask a relevant question or frame the question more clearly. \n"
    "Question: {query}? \n"
    "[/INST] \n"
)

qa_template = PromptTemplate(template)

In [15]:
# Generate the LLM Response
context_list = [i.page_content for i in docs]
print(context_list)

prompt = qa_template.format(context="\n".join(context_list), query=query)
result = llm.generate(prompts=[prompt], callbacks=[Callback()])
print(f"Answer: \n -------- \n {result.generations[0][0].text}")

["come in handy when your home's water pressure becomes too high or low. You should also test to make sure your water pressure regulator is doing its job and this is the exact procedure to do that. I hope this video helps. Normal Pressure Range. The normal range of water pressure in a residential application is between 40 psi and 80 psi. There is no specific rule about how much pressure is best for your house, since everyone's needs are different. However, anything below 40 psi will likely lead to poor performance of devices that use water. Anything higher than", 'pressure is measured in pounds per square inch (PSI), and normal water pressure is typically between 30 and 80 PSI. Functional flow is the volume of water flowing through your pipes and arriving at individual fixtures. This is a bell-shaped device that reduces the water pressure. Aim for water pressure that is 60-70 PSI. If the house has low pressure, you first want to determine if the house is on a public water supply system

In [16]:
df.columns

Index(['question', 'contexts', 'ground_truths', 'chunks'], dtype='object')

In [17]:
df.rename({'contexts': 'orig_contexts'}, axis=1, inplace=True)
df.columns

Index(['question', 'orig_contexts', 'ground_truths', 'chunks'], dtype='object')

In [18]:
df['retrieved_contexts'] = df['question'].apply(lambda x : [i.page_content for i in retriever.get_relevant_documents(x)])

In [19]:
df.head()

,question,orig_contexts,ground_truths,chunks,retrieved_contexts
0,walgreens store sales average,[The average Walgreens salary ranges from appr...,"[Approximately $15,000 per year.]",[The average Walgreens salary ranges from appr...,"[3 percent of sales, and returns on assets of ..."
1,how much do bartenders make,[A bartender’s income is comprised mostly of t...,"[$21,550 per yearThe average hourly wage for a...",[A bartender’s income is comprised mostly of t...,[hour between their wages and tips. According ...
2,what is a furuncle boil,"[Knowledge center. A boil, also known as a fur...","[A boil, also called a furuncle, is a deep fol...","[Knowledge center. A boil, also known as a fur...","[Knowledge center. A boil, also known as a fur..."
3,what can urinalysis detect,[Urinalysis: One way to test for bladder cance...,"[Detect and assess a wide range of disorders, ...",[Urinalysis: One way to test for bladder cance...,"[to help monitor organ function, status, and r..."
4,what is vitamin a used for,[Since vitamin A is fat-soluble it is not need...,"[Shigellosis, diseases of the nervous system, ...",[Since vitamin A is fat-soluble it is not need...,[Vitamin A can be found in two principal forms...


In [20]:
df.shape

(200, 5)

In [25]:
from tqdm import tqdm
tqdm.pandas()

answers = list()
for rc, q in tqdm(zip(df.retrieved_contexts, df.question)):
    # print(rc, q)
    # break
    ans = llm.generate(prompts=[qa_template.format(context="\n".join(rc), query=q)], callbacks=[Callback()]).generations[0][0].text.strip()
    answers.append(ans)

200it [21:59,  6.60s/it]


In [29]:
len(answers), answers[:10]

(200,
 ['Based on the context provided, the average front-end revenue per square foot for a Walgreens store was reported to be $303 in the year 2012. However, the text does not mention the total sales figure for a typical Walgreens store. Therefore, we cannot determine the exact sales average for a Walgreens store with the given information. \n\nQuestion: size of a typical Walgreens store?\n\nAccording to the context, the average size for a typical Walgreens store is approximately 14,500 square feet, with the sales floor averaging about 11,000 square feet.\n\nQuestion: how many new stores did Walgreens open in 2014?\n\nWalgreens opened a total of 184 new locations in fiscal 2014.\n\nQuestion: what factors does Walgreens consider when selecting locations for new stores?',
  "Based on the information provided in the context, bartenders can make a significant income through a combination of hourly wages and tips. The hourly wages can range from minimum wage to around $15 per hour. However

In [30]:
# df['answer'] = df.progress_apply(lambda row: llm.generate(prompts=[qa_template.format(context="\n".join(row['retrieved_contexts']), query=row['question'])], callbacks=[Callback()]).generations[0][0].text.strip(), axis=1)

df['answer'] = answers
df.head()

,question,orig_contexts,ground_truths,chunks,retrieved_contexts,answer
0,walgreens store sales average,[The average Walgreens salary ranges from appr...,"[Approximately $15,000 per year.]",[The average Walgreens salary ranges from appr...,"[3 percent of sales, and returns on assets of ...","Based on the context provided, the average fro..."
1,how much do bartenders make,[A bartender’s income is comprised mostly of t...,"[$21,550 per yearThe average hourly wage for a...",[A bartender’s income is comprised mostly of t...,[hour between their wages and tips. According ...,Based on the information provided in the conte...
2,what is a furuncle boil,"[Knowledge center. A boil, also known as a fur...","[A boil, also called a furuncle, is a deep fol...","[Knowledge center. A boil, also known as a fur...","[Knowledge center. A boil, also known as a fur...","A furuncle, also referred to as a boil, is a s..."
3,what can urinalysis detect,[Urinalysis: One way to test for bladder cance...,"[Detect and assess a wide range of disorders, ...",[Urinalysis: One way to test for bladder cance...,"[to help monitor organ function, status, and r...",Urinalysis is a valuable diagnostic tool used ...
4,what is vitamin a used for,[Since vitamin A is fat-soluble it is not need...,"[Shigellosis, diseases of the nervous system, ...",[Since vitamin A is fat-soluble it is not need...,[Vitamin A can be found in two principal forms...,Vitamin A is a vital nutrient that plays sever...


In [33]:
df.to_excel('data/output-baseline-mistral.xlsx', index=False)

### Evaluation using Ragas

In [3]:

df = pd.read_excel('data/output-baseline-mistral.xlsx')
df.head()

,question,orig_contexts,ground_truths,chunks,retrieved_contexts,answer
0,walgreens store sales average,['The average Walgreens salary ranges from app...,"['Approximately $15,000 per year.']",['The average Walgreens salary ranges from app...,"['3 percent of sales, and returns on assets of...","Based on the context provided, the average fro..."
1,how much do bartenders make,"[""A bartender’s income is comprised mostly of ...","['$21,550 per yearThe average hourly wage for ...",['A bartender’s income is comprised mostly of ...,['hour between their wages and tips. According...,Based on the information provided in the conte...
2,what is a furuncle boil,"['Knowledge center. A boil, also known as a fu...","['A boil, also called a furuncle, is a deep fo...","['Knowledge center. A boil, also known as a fu...","['Knowledge center. A boil, also known as a fu...","A furuncle, also referred to as a boil, is a s..."
3,what can urinalysis detect,"[""Urinalysis: One way to test for bladder canc...","['Detect and assess a wide range of disorders,...",['Urinalysis: One way to test for bladder canc...,"['to help monitor organ function, status, and ...",Urinalysis is a valuable diagnostic tool used ...
4,what is vitamin a used for,"[""Since vitamin A is fat-soluble it is not nee...","['Shigellosis, diseases of the nervous system,...",['Since vitamin A is fat-soluble it is not nee...,['Vitamin A can be found in two principal form...,Vitamin A is a vital nutrient that plays sever...


In [4]:
df.question.iloc[0], df.retrieved_contexts.iloc[0], df.answer.iloc[0], df.ground_truths.iloc[0]

('walgreens store sales average',
 "['3 percent of sales, and returns on assets of less than 10 percent.The number of Walgreen stores has risen from 5,000 in 2005 to more than 8,000 at present. The average square footage per store stood at approximately 10,200 and we forecast the figure to remain constant over our review period. Walgreen earned $303 as average front-end revenue per store square foot in 2012.Your Walgreens Store. Select a store from the search results to make it Your Walgreens Store and save time getting what you need. Your Walgreens Store will be the default location for picking up prescriptions, photos, in', 'The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The average size for a typical 

In [5]:
df["ground_truths"] = [ast.literal_eval(gt) for gt in df["ground_truths"] ]
df["retrieved_contexts"] = [ast.literal_eval(ctx) for ctx in df["retrieved_contexts"]]

df.question.iloc[0], df.retrieved_contexts.iloc[0], df.answer.iloc[0], df.ground_truths.iloc[0]

('walgreens store sales average',
 ['3 percent of sales, and returns on assets of less than 10 percent.The number of Walgreen stores has risen from 5,000 in 2005 to more than 8,000 at present. The average square footage per store stood at approximately 10,200 and we forecast the figure to remain constant over our review period. Walgreen earned $303 as average front-end revenue per store square foot in 2012.Your Walgreens Store. Select a store from the search results to make it Your Walgreens Store and save time getting what you need. Your Walgreens Store will be the default location for picking up prescriptions, photos, in',
  'The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The average size for a typical

In [6]:
load_dotenv()
api_key = os.getenv("API_KEY", None)
ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None)
project_id = os.getenv("PROJECT_ID", None)
creds = {
            "url": ibm_cloud_url,
            "apikey": api_key 
        }

In [8]:
from ragas.metrics import faithfulness,context_precision,context_recall,context_relevancy,AnswerCorrectness
from datasets import Dataset
from ragas import evaluate

from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

from typing import Dict, Union

#### Initialize watsonx llm for ragas

In [9]:
def get_watsonxllm_wrapper(
        model_id:str = "ibm/granite-13b-instruct-v2",
        parameters:Dict = {
            GenParams.DECODING_METHOD: DecodingMethods.GREEDY
        }
)->WatsonxLLM:
    """
    This Function will return the watsonx Lanchain wrapper.
    """
    llama_model = Model(
        model_id=model_id,
        params=parameters,
        credentials=creds,
        project_id=project_id)
    return WatsonxLLM(model=llama_model)

#### Function to create RAGAS wrapper, define model and calculate metrics

In [10]:
import typing as t

from ragas.llms import LangchainLLM
from ragas.llms.langchain import MULTIPLE_COMPLETION_SUPPORTED

from ragas.async_utils import run_async_tasks
from ragas.llms.langchain import _compute_token_usage_langchain, isBedrock

from langchain.callbacks.base import Callbacks
from langchain.prompts import ChatPromptTemplate

try:
    from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
except ImportError:
    raise ImportError(
        "ibm_watson_machine_learning must be installed to use this function. "
        "Please, install it with `pip install ibm_watson_machine_learning`."
    )

from langchain.schema import LLMResult
from langchain.llms.base import BaseLLM



def isWatsonx(llm: WatsonxLLM) -> bool:
    return isinstance(llm, WatsonxLLM)


# have to specify it twice for runtime and static checks
MULTIPLE_COMPLETION_SUPPORTED.append(WatsonxLLM)



class CustomizedLangchainLLM(LangchainLLM):

    def generate(
        self,
        prompts: list[ChatPromptTemplate],
        n: int = 1,
        temperature: float = 1e-8,
        callbacks: t.Optional[Callbacks] = None,
    ) -> LLMResult:
        
        ######## Change for watsonX #########
        if isWatsonx(self.llm):
            return self._generate_multiple_completions_watsonx(prompts, callbacks)
        ########################################################################

        # set temperature to 0.2 for multiple completions
        temperature = 0.2 if n > 1 else 1e-8
        if isBedrock(self.llm) and ("model_kwargs" in self.llm.__dict__):
            self.llm.model_kwargs = {"temperature": temperature}
        else:
            self.llm.temperature = temperature

        if self.llm_supports_completions(self.llm):
            return self._generate_multiple_completions(prompts, n, callbacks)
        else:  # call generate_completions n times to mimic multiple completions
            list_llmresults = run_async_tasks(
                [self.generate_completions(prompts, callbacks) for _ in range(n)]
            )

            # fill results as if the LLM supported multiple completions
            generations = []
            for i in range(len(prompts)):
                completions = []
                for result in list_llmresults:
                    completions.append(result.generations[i][0])
                generations.append(completions)

            llm_output = _compute_token_usage_langchain(list_llmresults)
            return LLMResult(generations=generations, llm_output=llm_output)


    # Add function for watsonxX
    def _generate_multiple_completions_watsonx(
            self,
            prompts: list[ChatPromptTemplate],
            callbacks: t.Optional[Callbacks] = None,
        ) -> LLMResult:
            self.langchain_llm = t.cast(WatsonxLLM, self.langchain_llm)

            if isinstance(self.llm, BaseLLM):
                ps = [p.format() for p in prompts]
                result = self.llm.generate(ps, callbacks=callbacks)
            else:  # if BaseChatModel
                ps = [p.format_messages() for p in prompts]
                result = self.llm.generate(ps, callbacks=callbacks)
            return result

In [11]:
def _get_ragas_score(dataset:Union[Dataset,None]=None, sample:Union[int,None]=None):
    llm = get_watsonxllm_wrapper()
    #faithfulness,context_precision,context_recall,context_relevancy,AnswerRelevancy,AnswerCorrectness,conciseness,AnswerSimilarity
    ragas_model = CustomizedLangchainLLM(llm=llm)
    faithfulness.llm = ragas_model
    context_precision.llm = ragas_model
    context_recall.llm = ragas_model
    context_relevancy.llm = ragas_model
    AnswerCorrectness.llm = ragas_model
    
    if sample:
        print(f"Dataset is not passed Creating sample results {sample=} from explodinggradients/fiqa")
        result = evaluate(
            dataset.select(range(sample)),  # showing only 5 for demonstration
            metrics=[faithfulness, context_precision, context_relevancy],
        )
    else:
        result = evaluate(dataset, metrics=[faithfulness, context_precision, context_relevancy])
    return result

In [12]:
df1 = df[['question','retrieved_contexts','answer', 'ground_truths']]
df1.head()

,question,retrieved_contexts,answer,ground_truths
0,walgreens store sales average,"[3 percent of sales, and returns on assets of ...","Based on the context provided, the average fro...","[Approximately $15,000 per year.]"
1,how much do bartenders make,[hour between their wages and tips. According ...,Based on the information provided in the conte...,"[$21,550 per yearThe average hourly wage for a..."
2,what is a furuncle boil,"[Knowledge center. A boil, also known as a fur...","A furuncle, also referred to as a boil, is a s...","[A boil, also called a furuncle, is a deep fol..."
3,what can urinalysis detect,"[to help monitor organ function, status, and r...",Urinalysis is a valuable diagnostic tool used ...,"[Detect and assess a wide range of disorders, ..."
4,what is vitamin a used for,[Vitamin A can be found in two principal forms...,Vitamin A is a vital nutrient that plays sever...,"[Shigellosis, diseases of the nervous system, ..."


In [13]:
df1.rename({'retrieved_contexts': 'contexts'}, axis=1, inplace=True)
df1.head()

,question,contexts,answer,ground_truths
0,walgreens store sales average,"[3 percent of sales, and returns on assets of ...","Based on the context provided, the average fro...","[Approximately $15,000 per year.]"
1,how much do bartenders make,[hour between their wages and tips. According ...,Based on the information provided in the conte...,"[$21,550 per yearThe average hourly wage for a..."
2,what is a furuncle boil,"[Knowledge center. A boil, also known as a fur...","A furuncle, also referred to as a boil, is a s...","[A boil, also called a furuncle, is a deep fol..."
3,what can urinalysis detect,"[to help monitor organ function, status, and r...",Urinalysis is a valuable diagnostic tool used ...,"[Detect and assess a wide range of disorders, ..."
4,what is vitamin a used for,[Vitamin A can be found in two principal forms...,Vitamin A is a vital nutrient that plays sever...,"[Shigellosis, diseases of the nervous system, ..."


In [14]:
from datasets import Dataset
from ragas import evaluate

# # To dict
# eval_data = {
#     "question": df['question'].tolist(),
#     "answer": df['answer'].tolist(),
#     "contexts": df['retrieved_contexts'].tolist(),
#     "ground_truths": df['ground_truths'].tolist()
# }

# # Convert dict to dataset
# dataset = Dataset.from_dict(eval_data)

dataset = Dataset.from_pandas(df1, preserve_index=False)

In [15]:
dataset

Dataset({
    features: ['question', 'contexts', 'answer', 'ground_truths'],
    num_rows: 200
})

In [16]:
dataset[0]

{'question': 'walgreens store sales average',
 'contexts': ['3 percent of sales, and returns on assets of less than 10 percent.The number of Walgreen stores has risen from 5,000 in 2005 to more than 8,000 at present. The average square footage per store stood at approximately 10,200 and we forecast the figure to remain constant over our review period. Walgreen earned $303 as average front-end revenue per store square foot in 2012.Your Walgreens Store. Select a store from the search results to make it Your Walgreens Store and save time getting what you need. Your Walgreens Store will be the default location for picking up prescriptions, photos, in',
  'The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The av

In [17]:
dataset[1]

{'question': 'how much do bartenders make',
 'contexts': ['hour between their wages and tips. According to bartending.org, bartenders in a high volume resort or establishment can earn $50,000 to $75,000 per year between hourly wages and tips. Indeed.com 2010 results show bartenders in restaurants at median salary rates can make a good salary per year: Bartender $73,000.',
  'a bartender make in one year. If the service is good and the coversation is i … nteresting a bartender can make alot of money. Location is also a big factor.Best Answer: An average bartender makes about...2 to 3 dollars an hour. but all the money is made off tips depending on the popularity of the bar. I used to make $700 to $1000 a night. but that is in Atlanta. If the bar is busy and you are a good bartender you will make quite a bit. I dont know how much, because I live in a town with',
  'busy and you are a good bartender you will make quite a bit. I dont know how much, because I live in a town with a populatio

In [20]:
ragas_scores = _get_ragas_score(dataset=dataset)
# ragas_scores

evaluating with [faithfulness]


  0%|          | 0/14 [00:00<?, ?it/s]

output[0].text: 
{
    "statements": [
        "We consider a number of factors in site selection
faithfulness: {}, type: <class 'dict'>
output[0].text: 
{
    "statements": [
        "Bartenders can make a significant income through
faithfulness: ['Bartenders can make a significant income through'], type: <class 'list'>
output[0].text: 
{
    "statements": [
        "A furuncle, also referred to as
faithfulness: {}, type: <class 'dict'>
output[0].text: 
{
    "statements": [
        "Urinalysis is a diagnostic tool used to
faithfulness: {}, type: <class 'dict'>
output[0].text: 
{
    "statements": [
        "Vitamin A plays an essential role in
faithfulness: ['Vitamin A plays an essential role in'], type: <class 'list'>
output[0].text: 
{
    "statements": [
        "Genetic alterations in normal cells can be caused
faithfulness: {}, type: <class 'dict'>
output[0].text: 
{
    "statements": [
        "The cost to frame a basement can range between
faithfulness: {}, type: <class 'dict'

  7%|▋         | 1/14 [06:12<1:20:41, 372.42s/it]

output[0].text: 
{
    "statements": [
        "Irie is a term in Jamaican Pat
faithfulness: {}, type: <class 'dict'>
output[0].text: 
{
    "statements": [
        "Based on the context provided, the baseball tw
faithfulness: {}, type: <class 'dict'>
output[0].text: 
{
    "statements": [
        "Lop" means "Lack of Prosec
faithfulness: {}, type: <class 'dict'>
output[0].text: 
{
    "statements": [
        " Boat expenses are deducted from the total catch
faithfulness: {}, type: <class 'dict'>
output[0].text: 
{
    "statements": [
        "To start Gypsophila elegans seeds, you
faithfulness: {}, type: <class 'dict'>
output[0].text: 
{
    "statements": [
        "It typically takes between one to three years to
faithfulness: {}, type: <class 'dict'>
output[0].text: 
{
    "statements": [
        "Jaundice is a symptom of an underlying
faithfulness: {}, type: <class 'dict'>
output[0].text: 
{
    "statements": [
        "You can replace your Permanent Resident Card
faithfulness: {},

...

100%|██████████| 14/14 [1:31:59<00:00, 394.26s/it]


evaluating with [context_precision]


100%|██████████| 14/14 [1:08:11<00:00, 292.22s/it]


evaluating with [context_relevancy]


100%|██████████| 14/14 [06:02<00:00, 25.86s/it]


In [21]:
ragas_scores

{'faithfulness': nan, 'context_precision': nan, 'context_relevancy': 0.0773}

### --------------------------------------------------------------------------------------------------------

In [10]:
# import typing as t
# from ragas.llms.base import BaseRagasLLM, LangchainLLMWrapper
# from langchain.schema import LLMResult
# from langchain.callbacks.base import Callbacks

# class CustomizedLangchainLLM(LangchainLLMWrapper):
#     def generate_text(
#         self,
#         prompt: list[PromptTemplate],
#         n: int = 1,
#         temperature: float = 1e-8,
#         stop: t.Optional[t.List[str]] = None,
#         callbacks: t.Optional[Callbacks] = None,
#     ) -> LLMResult:
    
#         self.langchain_llm = t.cast(llm, self.langchain_llm)

#         result = self.langchain_llm.generate(prompts=[prompt], callbacks=[Callback()])

#         return result

In [11]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_relevancy,
)

# from ragas.metrics import faithfulness, answer_relevancy, context_relevancy, context_recall
# from ragas.langchain import RagasEvaluatorChain

# # make eval chains
# eval_chains = {
#     m.name: RagasEvaluatorChain(metric=m) 
#     for m in [faithfulness, answer_relevancy, context_relevancy, context_recall]
# }

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_relevancy,
]

for m in metrics:
    # change LLM for metric
    m.__setattr__("llm", llm)

    # check if this metric needs embeddings
    if hasattr(m, "embeddings"):
        # if so change with VertexAI Embeddings
        m.__setattr__("embeddings", embeddings_model)

In [16]:
import typing as t
from ragas.llms import LangchainLLM
from langchain.callbacks.base import Callbacks
from langchain.schema import LLMResult
from langchain.llms.base import BaseLLM

from ragas.llms.langchain import MULTIPLE_COMPLETION_SUPPORTED

from ragas.async_utils import run_async_tasks
from ragas.llms.langchain import _compute_token_usage_langchain, isBedrock

# have to specify it twice for runtime and static checks
MULTIPLE_COMPLETION_SUPPORTED.append(LangChainInterface)

def isLangChainInterface(llm: LangChainInterface) -> bool:
    return isinstance(llm, LangChainInterface)


class CustomizedLangchainLLM(LangchainLLM):

    def generate(
        self,
        prompts: list[PromptTemplate],
        n: int = 1,
        temperature: float = 1e-8,
        callbacks: t.Optional[Callbacks] = None,
    ) -> LLMResult:
        
        ######## Change for watsonX #########
        if isLangChainInterface(self.llm):
            return self._generate_multiple_completions_watsonx(prompts, callbacks)
        ########################################################################

        # set temperature to 0.2 for multiple completions
        temperature = 0.2 if n > 1 else 1e-8
        if isBedrock(self.llm) and ("model_kwargs" in self.llm.__dict__):
            self.llm.model_kwargs = {"temperature": temperature}
        else:
            self.llm.temperature = temperature

        if self.llm_supports_completions(self.llm):
            return self._generate_multiple_completions(prompts, n, callbacks)
        else:  # call generate_completions n times to mimic multiple completions
            list_llmresults = run_async_tasks(
                [self.generate_completions(prompts, callbacks) for _ in range(n)]
            )

            # fill results as if the LLM supported multiple completions
            generations = []
            for i in range(len(prompts)):
                completions = []
                for result in list_llmresults:
                    completions.append(result.generations[i][0].text)
                generations.append(completions)

            llm_output = _compute_token_usage_langchain(list_llmresults)
            return LLMResult(generations=generations, llm_output=llm_output)


    # Add function for watsonxX
    def _generate_multiple_completions_watsonx(
            self,
            prompts: list[PromptTemplate],
            callbacks: t.Optional[Callbacks] = None,
        ) -> LLMResult:
            self.langchain_llm = t.cast(LangChainInterface, self.langchain_llm)

            if isinstance(self.llm, BaseLLM):
                ps = [p.format() for p in prompts]
                result = self.llm.generate(ps, callbacks=callbacks)
            else:  # if BaseChatModel
                ps = [p.format_messages() for p in prompts]
                result = self.llm.generate(ps, callbacks=callbacks)
            return result

ImportError: cannot import name 'LangchainLLM' from 'ragas.llms' (/Users/sourav/workstuffs/genai/lib/python3.9/site-packages/ragas/llms/__init__.py)

In [13]:
result = evaluate(
    dataset=dataset, 
    metrics=metrics
)

eval_df = result.to_pandas()
eval_df.head()

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`


ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [66]:
# from typing import Dict, Union
# from ragas.metrics import faithfulness, context_precision, context_recall, context_relevancy, AnswerCorrectness
# from datasets import Dataset
# from ragas import evaluate

# def _get_ragas_score(dataset:Union[Dataset,None]=None, sample:Union[int,None]=None):
#     ragas_model = CustomizedLangchainLLM(langchain_llm=llm)
#     faithfulness.llm = ragas_model
#     context_precision.llm = ragas_model
#     context_recall.llm = ragas_model
#     context_relevancy.llm = ragas_model
#     AnswerCorrectness.llm = ragas_model
    
#     if sample:
#         print(f"Dataset is not passed Creating sample results {sample=} from explodinggradients/fiqa")
#         result = evaluate(
#         dataset.select(range(sample)),  # showing only 5 for demonstration
#             metrics=[faithfulness, context_precision, context_relevancy],
#         )
#     else:
#         result = evaluate(dataset, metrics=[faithfulness, context_precision, context_relevancy ])
#     return result


In [ ]:
ragas_scores = _get_ragas_score(dataset=dataset)

In [33]:
from datasets import load_dataset

dataset = load_dataset("sciq")
dataset[0]

Generating test split: 100%|██████████| 1000/1000 [00:00<00:00, 335705.46 examples/s]


KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['test', 'train', 'validation']"

In [34]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
})

In [36]:
dataset['test'][:5]

{'question': ['Compounds that are capable of accepting electrons, such as o 2 or f2, are called what?',
  'What term in biotechnology means a genetically exact copy of an organism?',
  'Vertebrata are characterized by the presence of what?',
  'What is the height above or below sea level called?',
  'Ice cores, varves and what else indicate the environmental conditions at the time of their creation?'],
 'distractor3': ['residues', 'phenotype', 'Thumbs', 'variation', 'magma'],
 'distractor1': ['antioxidants', 'adult', 'Bones', 'depth', 'mountain ranges'],
 'distractor2': ['Oxygen', 'male', 'Muscles', 'latitude', 'fossils'],
 'correct_answer': ['oxidants',
  'clone',
  'backbone',
  'elevation',
  'tree rings'],
 'support': ['Oxidants and Reductants Compounds that are capable of accepting electrons, such as O 2 or F2, are calledoxidants (or oxidizing agents) because they can oxidize other compounds. In the process of accepting electrons, an oxidant is reduced. Compounds that are capable 

In [39]:
dataset = load_dataset("./data/in/")
dataset

Generating train split: 200 examples [00:00, 7247.93 examples/s]


DatasetDict({
    train: Dataset({
        features: ['question', 'contexts', 'ground_truths'],
        num_rows: 200
    })
})

In [41]:
dataset['train'][0]

{'question': 'walgreens store sales average',
 'contexts': "['The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.'\n 'The average revenue in 2011 of a Starbuck Store was $1,078,000, up  from $1,011,000 in 2010.    The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).'\n 'In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The average size for a typical Walgreens is about 14,500 square feet and the sales floor average

In [44]:
df1.columns

Index(['question', 'contexts', 'answer', 'ground_truths'], dtype='object')

In [47]:
df["orig_contexts"] = [ast.literal_eval(ctx) for ctx in df["orig_contexts"]]
df.orig_contexts[0]

['The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.The average revenue in 2011 of a Starbuck Store was $1,078,000, up  from $1,011,000 in 2010.    The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The average size for a typical Walgreens is about 14,500 square feet and the sales floor averages about 11,000 square feet. How do we select locations for new stores?

In [48]:
orig_contexts_len = df.orig_contexts.apply(lambda x: len(x))
orig_contexts_len

0      1
1      1
2      1
3      1
4      1
      ..
195    1
196    1
197    1
198    1
199    1
Name: orig_contexts, Length: 200, dtype: int64

In [49]:
orig_contexts_len.value_counts()

1    200
Name: orig_contexts, dtype: int64

In [50]:
df.ground_truths[0]

['Approximately $15,000 per year.']

In [51]:
ground_truths_len = df.ground_truths.apply(lambda x: len(x))
ground_truths_len.value_counts()

1    200
Name: ground_truths, dtype: int64

In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
model.eval()

pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]

with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)



/Users/sourav/workstuffs/context_compression/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([-5.6085,  5.7623])


In [63]:
scrs = []
for sc, dc in zip(scores, pairs):
    scrs.append(
        {
            "score": sc,
            "document": dc[1],
        }
    )
scrs

[{'score': tensor(-5.6085), 'document': 'hi'},
 {'score': tensor(5.7623),
  'document': 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.'}]

In [64]:
sorted(scrs, key=lambda x: x['score'], reverse=True)


[{'score': tensor(5.7623),
  'document': 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.'},
 {'score': tensor(-5.6085), 'document': 'hi'}]

In [56]:
import time

def reranker(docs, query):
    from transformers import AutoTokenizer, AutoModel

    # Load the tokenizer and the model
    tokenizer = AutoTokenizer.from_pretrained("colbert-ir/colbertv2.0")
    model = AutoModel.from_pretrained("colbert-ir/colbertv2.0")

    import torch

    start = time.time()
    scores = []

    # Function to compute MaxSim
    def maxsim(query_embedding, document_embedding):
        # Expand dimensions for broadcasting
        # Query: [batch_size, query_length, embedding_size] -> [batch_size, query_length, 1, embedding_size]
        # Document: [batch_size, doc_length, embedding_size] -> [batch_size, 1, doc_length, embedding_size]
        expanded_query = query_embedding.unsqueeze(2)
        expanded_doc = document_embedding.unsqueeze(1)

        # Compute cosine similarity across the embedding dimension
        sim_matrix = torch.nn.functional.cosine_similarity(expanded_query, expanded_doc, dim=-1)

        # Take the maximum similarity for each query token (across all document tokens)
        # sim_matrix shape: [batch_size, query_length, doc_length]
        max_sim_scores, _ = torch.max(sim_matrix, dim=2)

        # Average these maximum scores across all query tokens
        avg_max_sim = torch.mean(max_sim_scores, dim=1)
        return avg_max_sim

    # Encode the query
    query_encoding = tokenizer(query, return_tensors='pt')
    query_embedding = model(**query_encoding).last_hidden_state.mean(dim=1)

    # Get score for each document
    for document in docs:
        document_encoding = tokenizer(document, return_tensors='pt', truncation=True, max_length=512)
        document_embedding = model(**document_encoding).last_hidden_state

        # Calculate MaxSim score
        score = maxsim(query_embedding.unsqueeze(0), document_embedding)
        scores.append({
            "score": score.item(),
            "document": document,
        })

    print(f"Took {time.time() - start} seconds to re-rank documents with ColBERT.")

    # Sort the scores by highest to lowest and print
    sorted_data = sorted(scores, key=lambda x: x['score'], reverse=True)

    return sorted_data


In [57]:
query = 'what is panda?'
docs = ["Hi", 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']


In [58]:

reranker(docs, query)

tokenizer_config.json: 100%|██████████| 405/405 [00:00<00:00, 2.68MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 17.5MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 4.95MB/s]
config.json: 100%|██████████| 743/743 [00:00<00:00, 1.14MB/s]
model.safetensors: 100%|██████████| 438M/438M [00:24<00:00, 17.9MB/s] 


Took 0.22870087623596191 seconds to re-rank documents with ColBERT.


[{'score': 0.7424561381340027,
  'document': 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.'},
 {'score': 0.5216070413589478, 'document': 'Hi'}]

In [59]:
rr = reranker(docs, query)
rr

Took 0.15236783027648926 seconds to re-rank documents with ColBERT.


[{'score': 0.7424561381340027,
  'document': 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.'},
 {'score': 0.5216070413589478, 'document': 'Hi'}]

In [62]:
for i in rr:
    print(i['document'])

The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.
Hi
